## 词向量模型

In [1]:
#TODO：使用语料库训练
import json
import numpy as np
import re
import jieba

In [2]:
with open('../工单数据/2018年数据.json', encoding='utf-8') as f:
    data = json.load(f)

data = np.array(data)
X = data[:, 0]
Y = data[:, 1]

In [3]:
print(len(Y), len(X))

112028 112028


In [4]:
# 删去无标签数据
none_labels = []
for i in range(Y.shape[0]):
    if not Y[i]:
        # print(X[i])
        # print('*'*80)
        none_labels.append(i)

Y = np.delete(Y, none_labels)
X = np.delete(X, none_labels)

In [5]:
print(len(Y), len(X))

111814 111814


In [6]:
# 删去#号开头的文本和空文本
invaild_x = []
for i in range(X.shape[0]):
    if not X[i] or X[i].startswith('#'):
        invaild_x.append(i)

Y = np.delete(Y, invaild_x)
X = np.delete(X, invaild_x)

In [7]:
print(len(Y), len(X))

109580 109580


In [8]:
# 删去废单和回访工单
discard_x = []
for i in range(X.shape[0]):
    if X[i].strip().startswith('废单') or X[i].strip().startswith('回访工单'):
        discard_x.append(i)

Y = np.delete(Y, discard_x)
X = np.delete(X, discard_x)

In [9]:
print(len(Y), len(X))

104233 104233


In [10]:
# 删去非受理范围工单
non_acceptable = np.nonzero(Y[:] == '非受理范围')
Y = np.delete(Y, non_acceptable)
X = np.delete(X, non_acceptable)

In [11]:
# 有的无效电话不属于非受理范围
invaild_p = []
for i in range(X.shape[0]):
    if X[i].startswith('无效电话'):
        invaild_p.append(i)

Y = np.delete(Y, invaild_p)
X = np.delete(X, invaild_p)

In [12]:
print(len(Y), len(X))

100155 100155


In [13]:
# 加载停用词表
def get_stop_words():
    """停用词列表"""
    stop_words = []
    with open('./static/停用词.txt', encoding='utf-8-sig') as f:
        lines = f.readlines()
        
    for line in lines:
        stop_words.append(line.strip())
    
    return stop_words

s_words = get_stop_words()

In [14]:
import string


def judge(word):
    """判断是否由英文字母加数字组成"""
    for w in word:
        if w not in string.printable:
            return True
    
    return False  

In [16]:
#数据集预处理
jieba.load_userdict('./static/my_dict.txt')  # 加载分词词典

data_list = []
revisit_x = []  # 删除回访工单

for i in range(X.shape[0]):
    x = X[i]
    # 处理特殊工单:催单、补单、撤单、撤销
    if x.strip().startswith(('催单', '补单','撤单','撤销', '催办','转接')): 
        quert_l = re.split(r'\n+',x.strip())  # 以回车符分割字符串

        if len(quert_l) == 3:  # 催单、问题描述：、来电目的：
            q = re.findall('：(.*)', quert_l[1])[0].strip() # 匹配问题描述后文本
        
        elif len(quert_l) == 1:# 催单：  
            q = re.findall('：(.*)', quert_l[0])[0].strip() # 匹配冒号后文本
        
        else: #催单:\n
            q = quert_l[1].strip()
    
    else:  # 处理正常工单
        quert_l = re.split(r'\n+',x.strip())  # 以回车符分割字符串
        match_l = re.findall('：(.*)', quert_l[0]) # 匹配冒号后文本;1.问题描述：
        if not match_l:
            q = quert_l[0].strip()
        else:
            q = match_l[0].strip()
        
        
    # 分词
    seg_sentence = jieba.cut(q, cut_all=False)
    tmp_l = []

    # 遍历停用词表,去停用词
    for w in seg_sentence:
        if w not in s_words and len(w) >= 2 and not w.isdigit() and judge(w):  # TODO:判断词是否为小数
            tmp_l.append(w)

    if not tmp_l: # 工单是便民服务回访、工单回访;删去工单
        revisit_x.append(i)
#         print(x)
#         print('切分后',q)
#         print('*'*50)

    data_list.append(tmp_l)


data_list = np.array(data_list)

data_list = np.delete(data_list, revisit_x)
Y = np.delete(Y, revisit_x)

1.问题描述：市民反映铜山区相关问题，咨询问题处理进度。
2.来电目的：咨询问题处理进度。
切分后 市民反映铜山区相关问题，咨询问题处理进度。
**************************************************
市民前期反映国旅旅行社欺诈消费者的问题，现来电要求催单（前期工单号：XZ201804130533）。
切分后 XZ201804130533）。
**************************************************
补充：
市民前期反映洒水车在洒水作业时不避让行人，现来电补充信息（前期工单号：XZ201805120064），2018年5月14日，市民再次遇见该洒水车，再次找到司机，该洒水车司机态度非常恶劣，明确要求该洒水车司机给其道歉。且市民表示洒水车属于中治交通公司。
切分后 
**************************************************
1.问题描述：市民反映铜山区问题。
2.来电目的：希望相关部门核实处理。
切分后 市民反映铜山区问题。
**************************************************
便民服务回访：Tel1803010005 。
切分后 Tel1803010005 。
**************************************************
1.问题描述：市民咨询前期反映问题的处理进度。
2.来电目的：咨询前期反映问题的处理进度。
切分后 市民咨询前期反映问题的处理进度。
**************************************************
工单回访：XZ201803020847。
切分后 XZ201803020847。
**************************************************
市民前期反映国旅旅行社欺诈消费者的问题，现来电要求催单（前期工单号：XZ201804130533）。
切分后 XZ201804130533）。
**************************************************
市民咨询其孩子社保卡制卡状态（身份证号码：32030420040131042

In [44]:
print(len(data_list), len(Y))        

100093 100093


In [115]:
# jieba.load_userdict('./12345analysis/my_dict.txt')

# res = []

# for x in X:
#     if not x.strip().startswith('催单') and not x.strip().startswith('补单') and not x.strip().startswith('撤单'):
#         quert_l = re.split(r'\n+',x.strip())
#         match_l = re.findall('：(.*)', quert_l[0])
#         if not match_l:
#             res.append(quert_l[0])
#         else:
#             res.append(re.findall('：(.*)', quert_l[0])[0].strip())

#     if x.strip().startswith('催单') or x.strip().startswith('补单') or x.strip().startswith('撤单'):
#         r = re.split(r'\n+',x.strip())
#         if len(r) == 3:  # 有问题描述、来电目的
#             t = re.findall('：(.*)', r[1])[0].strip()
#         if len(r) == 1:
#             t = re.findall('：(.*)', r[0])[0].strip()
#         if len(r) == 5:
#             t = r[1].strip()
#             print(x)
#             print('*'*80)
            
#         seg = jieba.cut(t, cut_all=False)
#         tmp_l = []
#         for w in seg:
#             if w not in s_words and len(w) >= 2 and not w.isdigit() and judge(w):
#                 tmp_l.append(w)

#         res.append(tmp_l)

催单：
1、问题描述：服务对象表示2018年3月12日了解到徐州市丰县师寨镇李庄村后中队将进行拆迁，且现已进行评估，自己没有签订拆迁协议。
2、诉求目的：服务对象希望相关部门帮助协调不要对此处进行拆迁
3、事发时间：2018年3月12日
4、详细地址：徐州市丰县师寨镇李庄村后中队
********************************************************************************


In [19]:
# 保存语料
# with open('810corpus.json', 'w', encoding='utf-8') as f:
#     json.dump(data_list.tolist(), f, ensure_ascii=False)

In [20]:
# word2vec训练词向量
from gensim.models import Word2Vec

F:\anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [21]:
model = Word2Vec(data_list, size=100, window=5, min_count=5)

In [22]:
model.wv['公积金']

array([-1.08414829,  0.54210341, -0.4031122 ,  0.48025766, -1.14444888,
       -1.92862988,  0.00427932, -0.61854035, -1.07644463,  0.64203483,
       -0.88129771, -0.3059116 , -0.49983203, -1.67840242,  1.10129213,
        2.40429926, -1.20147848, -1.85267937,  0.28966635, -0.53846055,
        0.68714809,  0.53866065,  0.60825676, -2.78674316,  0.17950872,
        0.67289668, -0.82658482,  2.21098399, -1.97404611, -0.8330397 ,
       -1.08199215,  0.35954303,  0.70492953,  1.46160269,  0.60551614,
       -0.76757348,  0.3890844 ,  0.41197038,  0.60996038, -0.43112034,
        0.15025881, -1.10110569,  0.22174355, -0.66210145,  1.14905047,
       -0.18863261,  1.30557764,  1.45692301,  1.43132293, -1.46393394,
       -1.23628449,  0.18532652, -0.91341114,  1.12808692, -3.56868792,
       -2.15043497,  0.76306701,  1.49458456,  1.85907793,  2.34432125,
        1.27395403, -1.47399116,  1.67445409, -2.58999062,  1.13852239,
        0.10911949,  1.58281457, -0.23516248, -1.46640503, -3.20

In [23]:
model.wv.most_similar('公积金', topn=20)

[('住房', 0.8258473873138428),
 ('组合型', 0.7659595012664795),
 ('管理中心', 0.7514011263847351),
 ('贷款', 0.7482126951217651),
 ('可否', 0.7480394840240479),
 ('多长时间', 0.7460739612579346),
 ('还款', 0.743374228477478),
 ('商业贷款', 0.7430015802383423),
 ('选择', 0.7415223121643066),
 ('担保人', 0.7392716407775879),
 ('多久', 0.7376024723052979),
 ('买房', 0.7362715005874634),
 ('模式', 0.7346478700637817),
 ('提取', 0.7344260215759277),
 ('年限', 0.7318391799926758),
 ('偿还', 0.7292807102203369),
 ('填写', 0.7276386022567749),
 ('县区', 0.7269107103347778),
 ('配偶', 0.7268178462982178),
 ('几次', 0.7178781032562256)]

In [24]:
model.wv.similarity('转接', '12345热线')

0.29624555

In [25]:
model.save('88model')

In [26]:
model = Word2Vec.load('88model')

In [30]:
# 训练好的词向量数组
model.wv.vectors.shape

(10407, 100)

In [31]:
# 建立字典，key为单词，value为单词的词向量数组
words = model.wv.vocab.keys()
w2c_dic = dict()

for w in words:
    warray = model.wv.get_vector(w)
    w2c_dic[w] = warray

In [32]:
w2c_dic['公积金']

array([-1.08414829,  0.54210341, -0.4031122 ,  0.48025766, -1.14444888,
       -1.92862988,  0.00427932, -0.61854035, -1.07644463,  0.64203483,
       -0.88129771, -0.3059116 , -0.49983203, -1.67840242,  1.10129213,
        2.40429926, -1.20147848, -1.85267937,  0.28966635, -0.53846055,
        0.68714809,  0.53866065,  0.60825676, -2.78674316,  0.17950872,
        0.67289668, -0.82658482,  2.21098399, -1.97404611, -0.8330397 ,
       -1.08199215,  0.35954303,  0.70492953,  1.46160269,  0.60551614,
       -0.76757348,  0.3890844 ,  0.41197038,  0.60996038, -0.43112034,
        0.15025881, -1.10110569,  0.22174355, -0.66210145,  1.14905047,
       -0.18863261,  1.30557764,  1.45692301,  1.43132293, -1.46393394,
       -1.23628449,  0.18532652, -0.91341114,  1.12808692, -3.56868792,
       -2.15043497,  0.76306701,  1.49458456,  1.85907793,  2.34432125,
        1.27395403, -1.47399116,  1.67445409, -2.58999062,  1.13852239,
        0.10911949,  1.58281457, -0.23516248, -1.46640503, -3.20

In [41]:
# 词条未出现过，随机生成词向量。
# 词向量累加求平均值作为文本表示
docs2vec = []
key_l = w2c_dic.keys()
for text in data_list:
    d2v = np.zeros((100,))
    length = len(text)

    for w in text:
        if w in key_l:
            d2v += w2c_dic[w]

    docs2vec.append(d2v/length)

np.array(docs2vec).shape

(100093, 100)

In [42]:
from sklearn.ensemble import RandomForestClassifier

class Textclassify:
    def __init__(self, X, Y, stop_words):
        self.X = X
        self.Y = Y
        self.s_words = stop_words

    def cross_validation(self):
        """交叉验证，90%数据用于训练，10%数据用于测试"""
        data = list(zip(self.X, self.Y))
        sep = int(len(data) * 0.9)
        np.random.shuffle(data)  # 数据乱序

        train_data = data[:sep]
        test_data = data[sep:]

        X_train, Y_train = zip(*train_data)
        X_test, Y_test = zip(*test_data)

        return X_train, Y_train, X_test, Y_test

    def classify(self):
        """分类"""
        acc_li = []
        clf = RandomForestClassifier(n_estimators=100)  # 参数设置
        
        for i in range(10):
            X_train, Y_train, X_test, Y_test = self.cross_validation()
            clf.fit(X_train, Y_train)
            acc = clf.score(X_test, Y_test)
            print(acc)
            acc_li.append(acc)
        
        return acc_li

In [43]:
tc = Textclassify(docs2vec, Y, s_words)
acc = tc.classify()

0.432926829268
0.429328268693
0.4306277489
0.419732107157
0.429128348661
0.432526989204
0.429228308677
0.430827668932
0.435725709716
0.437824870052
